In [ ]:
# |default_exp overlay.round

In [ ]:
# |export
import dash_bootstrap_components as dbc
from aifront.overlay.default import background_color, table
from aifront.overlay.medal import pb_with_icon, racetime2str
from dash import html

In [ ]:
# |exporti
def diff_span(diff):
    return html.Span(
        diff, className="ms-1", style={"font-size": ".7em", "color": "lightskyblue"}
    )


def finish_with_diff(medals, best, previous_best=None, no_respawn=False):
    finish = pb_with_icon(medals, best, no_respawn)
    key = "no_respawn_finish_racetime" if no_respawn else "finish_racetime"
    # if current round or first round
    if previous_best is None or previous_best["total_reward"] is None:
        return finish
    if best.get(key, None) is not None:
        if previous_best.get(key, None) is not None:
            if best[key] != previous_best[key]:
                diff = previous_best[key] - best[key]
                return [finish, diff_span(f"-{racetime2str(diff)}")]
        else:
            return [finish, diff_span("new")]
    elif best["progress"] != previous_best["progress"]:
        diff = best["progress"] - previous_best["progress"]
        return [finish, diff_span(f"+{diff*100:.1f} %")]
    return finish


def reward_with_diff(best, previous_best=None, no_respawn=False):
    key = "no_respawn_reward" if no_respawn else "total_reward"
    reward = f"{best[key]:.1f}"
    # if current round or first round
    if previous_best is None or previous_best["total_reward"] is None:
        return reward
    if previous_best[key] != best[key]:
        diff = best[key] - previous_best[key]
        return [reward, diff_span(f"+{diff:.1f}")]
    return reward


summary_metrics = [
    "Finish",
    "Reward",
    "Respawn",
    "No respawn finish",
    "No respawn reward",
]


def summary(medals, best, previous_best=None):
    return (
        finish_with_diff(medals, best, previous_best, False),
        reward_with_diff(best, previous_best, False),
        best.get("total_respawn_count", "-"),
        finish_with_diff(medals, best, previous_best, True),
        reward_with_diff(best, previous_best, True),
    )

In [ ]:
# |export
def round_overlay(data):
    ss = data["state"]["session_scheduler"]
    round_name = html.P(
        f"Session {ss['session']} - Round {ss['round']} / {ss['nb_episode']}",
        className="p-2",
    )
    headers = [
        "Metric",
        "Round",
        f"Session {ss['session']} best",
        "Map overall best",
    ]
    medals = data["config"]["map"]["medals"]
    session_stats = data["stats"]["sessions"][str(data["stats"]["current_session"])]
    rows = zip(
        summary_metrics,
        summary(medals, data["info"]),
        summary(medals, session_stats["best"], data["previous_session_best"]),
        summary(
            medals, data["stats"]["overall"]["best"], data["previous_overall_best"]
        ),
    )
    return dbc.Badge(
        [round_name, table(headers, rows)],
        color=background_color,
        style={"border-radius": "1rem"},
    )

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()